In [1]:
!pip install libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 42.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# --- Step 1: Install required libraries ---
!pip install requests tqdm -q

# --- Step 2: The Main Application Code ---
import libtorrent as lt
import time
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from tqdm.notebook import tqdm

# --- Configuration ---
SAVE_PATH = "/content/drive/My Drive/Torrents"
SESSION_SAVE_PATH = os.path.join(SAVE_PATH, ".session")

# --- Setup ---
os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(SESSION_SAVE_PATH, exist_ok=True)

# --- Initialize Session & Widgets ---
print("Initializing torrent session...")
ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

active_handles = []
MONITORING_IN_PROGRESS = False

# --- Core Functions ---
def save_session_state():
    with open(os.path.join(SESSION_SAVE_PATH, "session.dat"), "wb") as f:
        f.write(lt.bencode(ses.save_state()))

def format_speed(speed_bytes):
    if speed_bytes < 1024: return f"{speed_bytes:.1f} B/s"
    if speed_bytes < 1024**2: return f"{speed_bytes/1024:.1f} KB/s"
    if speed_bytes < 1024**3: return f"{speed_bytes/1024**2:.1f} MB/s"
    return f"{speed_bytes/1024**3:.1f} GB/s"

def monitor_downloads(handle_list):
    global MONITORING_IN_PROGRESS
    MONITORING_IN_PROGRESS = True
    progress_bars = {}

    while any(h.is_valid() and not h.status().is_seeding for h in handle_list):
        # Create a copy to iterate over, as we might remove items from the original list
        for handle in list(handle_list):
            if not handle.is_valid():
                handle_list.remove(handle)
                continue

            if not handle.has_metadata():
                print(f"Waiting for metadata for a torrent...")
                time.sleep(0.5)
                continue

            s = handle.status()
            info_hash = str(handle.info_hash())

            if info_hash not in progress_bars:
                progress_bars[info_hash] = tqdm(
                    total=s.total_wanted, desc=handle.name()[:40], unit='B',
                    unit_scale=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
                )

            bar = progress_bars[info_hash]
            bar.n = s.total_done
            bar.set_postfix_str(f"↓{format_speed(s.download_rate)} ↑{format_speed(s.upload_rate)} | Peers: {s.num_peers}")
            bar.refresh()

            if s.is_seeding:
                if not bar.disable:
                    bar.set_postfix_str("Download Complete. Seeding...")
                    bar.close()
                # *** FIX *** This is critical for preventing the loop from getting stuck
                handle_list.remove(handle)

        ses.post_torrent_updates()
        save_session_state()
        time.sleep(1)

    print("\n--- All Active Downloads Complete! ---")
    for bar in progress_bars.values():
        if not bar.disable: bar.close()
    save_session_state()
    MONITORING_IN_PROGRESS = False

def download_torrent(link, handle_list):
    params = {'save_path': SAVE_PATH}
    try:
        if link.startswith("magnet:"): handle = lt.add_magnet_uri(ses, link, params)
        elif link.startswith("http"):
            info = lt.torrent_info(requests.get(link, timeout=10).content)
            handle = ses.add_torrent({'ti': info, 'save_path': SAVE_PATH})
        else:
            info = lt.torrent_info(link)
            handle = ses.add_torrent({'ti': info, 'save_path': SAVE_PATH})
        handle_list.append(handle)
        return handle
    except Exception as e:
        print(f"Error adding torrent: {e}")
        return None

def on_start_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        link = link_input.value
        if not link:
            print("Error: The text box is empty.")
            return
        link_input.value = ""
        print("Adding torrent to the session...")
        handle = download_torrent(link, active_handles)
        if handle and not MONITORING_IN_PROGRESS:
            monitor_downloads(active_handles)
        elif handle:
            print(f"Added new torrent to the queue.")

# --- UI and Main Logic ---
link_input = widgets.Text(value='', placeholder='Paste magnet, URL, or torrent path here', description='Source:', disabled=False, layout=widgets.Layout(width='80%'))
start_button = widgets.Button(description='Start Download', disabled=False, button_style='success', tooltip='Click to start the download', icon='download')
output_area = widgets.Output()
start_button.on_click(on_start_button_clicked)

# --- Main Script Execution ---
clear_output()
print("🚀 Torrent Downloader is Ready 🚀")
print("1. Make sure you have mounted your Google Drive.")
print(f"2. Files will be saved to: {SAVE_PATH}")

# *** FIX *** New section to explicitly show resumable torrents
try:
    with open(os.path.join(SESSION_SAVE_PATH, "session.dat"), "rb") as f:
        ses.load_state(f.read())
        print("\n✅ Previous session state loaded successfully.")
        resumed_handles = ses.get_torrents()
        if resumed_handles:
            print("--- Found Resumable Torrents ---")
            active_handles.extend(resumed_handles)
            for handle in active_handles:
                # Wait for metadata to print the name
                if not handle.has_metadata():
                    print("Getting metadata for a resumed torrent...")
                    time.sleep(1)
                if handle.has_metadata():
                    print(f"  ▶️ Resuming: {handle.name()}")
                else:
                    print(f"  ▶️ Resuming torrent with hash: {handle.info_hash()}")
            print("--------------------------------")
except Exception:
    print("\nℹ️ No previous session found. Starting fresh.")

# Display the UI
print("\n3. Paste a source link below and click 'Start Download'.")
display(link_input, start_button, output_area)

# Automatically start monitoring if we found resumable torrents
if active_handles and not MONITORING_IN_PROGRESS:
    with output_area:
        print("Automatically starting monitor for resumed torrents...")
        monitor_downloads(active_handles)

🚀 Torrent Downloader is Ready 🚀
1. Make sure you have mounted your Google Drive.
2. Files will be saved to: /content/drive/My Drive/Torrents

✅ Previous session state loaded successfully.

3. Paste a source link below and click 'Start Download'.


Text(value='', description='Source:', layout=Layout(width='80%'), placeholder='Paste magnet, URL, or torrent p…

Button(button_style='success', description='Start Download', icon='download', style=ButtonStyle(), tooltip='Cl…

Output()